In [ ]:
#importing libraries
import pandas as pd
import os
from sklearn import tree
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.tree import export_text
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
import seaborn as sn
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [ ]:
#import the dataset - May 3
df_full = pd.read_csv('may3fullnotnormalized.csv')

In [ ]:
#print head
df_full.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,file,labels
0,31.979785,31.974471,31.971309,31.961714,31.957032,31.953863,31.961393,31.968252,31.970117,31.964835,...,31.877288,31.885947,31.890582,31.903106,31.871349,31.898401,31.881826,31.882834,feb7,L_Proline
1,31.877456,31.852502,31.881129,31.869941,31.875711,31.893926,31.896504,31.873854,31.890934,31.900153,...,31.829525,31.836288,31.859889,31.847140,31.853526,31.823883,31.824403,31.805916,feb7,L_Proline
2,31.793558,31.797328,31.809885,31.805699,31.806156,31.807924,31.811702,31.805596,31.808508,31.822106,...,31.794167,31.800754,31.795079,31.815912,31.805555,31.826588,31.813639,31.803027,feb7,L_Proline
3,31.777048,31.767853,31.802315,31.790646,31.771182,31.765563,31.781762,31.812254,31.789830,31.822890,...,31.792999,31.790381,31.786940,31.792214,31.795751,31.795672,31.807004,31.809877,feb7,L_Proline
4,31.790894,31.775576,31.792422,31.769077,31.768876,31.745612,31.775135,31.759473,31.785412,31.789453,...,31.779601,31.770510,31.775127,31.781938,31.806588,31.803418,31.778440,31.762858,feb7,L_Proline


In [ ]:
#print shape
df_full.shape

(679, 74)

In [ ]:
#print unique labels
df_full['labels'].unique()

array(['L_Proline', 'L_Lysine', 'L_Glutamine', 'L_Arginine', 'L_Serine',
       'L_Histidine', 'L_Threonine', 'L_Asparagine', 'L_Phenylalanine',
       'L_Alanine', 'L_Glutamate', 'L_Methionine', 'L_Cystine',
       'L_Valine', 'L_Isoleucine', 'L_Tryptophan', 'L_Leucine'],
      dtype=object)

In [ ]:
#make a copy of the dataframe inot data_act
data_act = df_full.iloc[:,:74]
data_act.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,file,labels
0,31.979785,31.974471,31.971309,31.961714,31.957032,31.953863,31.961393,31.968252,31.970117,31.964835,...,31.877288,31.885947,31.890582,31.903106,31.871349,31.898401,31.881826,31.882834,feb7,L_Proline
1,31.877456,31.852502,31.881129,31.869941,31.875711,31.893926,31.896504,31.873854,31.890934,31.900153,...,31.829525,31.836288,31.859889,31.847140,31.853526,31.823883,31.824403,31.805916,feb7,L_Proline
2,31.793558,31.797328,31.809885,31.805699,31.806156,31.807924,31.811702,31.805596,31.808508,31.822106,...,31.794167,31.800754,31.795079,31.815912,31.805555,31.826588,31.813639,31.803027,feb7,L_Proline
3,31.777048,31.767853,31.802315,31.790646,31.771182,31.765563,31.781762,31.812254,31.789830,31.822890,...,31.792999,31.790381,31.786940,31.792214,31.795751,31.795672,31.807004,31.809877,feb7,L_Proline
4,31.790894,31.775576,31.792422,31.769077,31.768876,31.745612,31.775135,31.759473,31.785412,31.789453,...,31.779601,31.770510,31.775127,31.781938,31.806588,31.803418,31.778440,31.762858,feb7,L_Proline


In [ ]:
data_act.tail()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,file,labels
674,37.815687,37.800834,37.803323,37.807549,37.845211,37.791358,37.803139,37.837304,37.833711,37.830318,...,37.864212,37.882058,37.892543,37.880778,37.900849,37.886396,37.885324,37.867085,may3,L_Arginine
675,37.835352,37.835408,37.862491,37.861515,37.841402,37.841402,37.842315,37.850077,37.865932,37.866044,...,37.900018,37.911045,37.912006,37.914976,37.925220,37.930110,37.899866,37.891758,may3,L_Arginine
676,37.886228,37.878585,37.893999,37.891390,37.868309,37.885620,37.892871,37.885012,37.908845,37.919881,...,37.981658,37.978129,37.986020,37.991662,37.990726,37.992743,37.990782,37.986723,may3,L_Arginine
677,37.985556,37.980241,37.966043,37.973630,37.973383,37.971470,37.977840,37.988093,37.984275,37.992134,...,37.971430,37.976809,37.975200,37.982426,37.980178,37.975992,37.975616,37.969709,may3,L_Arginine
678,37.965813,37.968117,37.959857,37.960770,37.947293,37.968557,37.962323,37.978968,37.979361,37.985467,...,37.922147,37.917168,37.911654,37.926524,37.946237,37.961217,37.963563,37.948253,may3,L_Arginine


In [ ]:
#linear baseline correction
import math
def linear(dfl):
  #print(dfl)
  b = dfl[0]
  a = ((dfl[-1] - dfl[0])/(355*math.pi/180))
  angle=list(range(0,360,5))
  for i in range(len(dfl)):
    dfl[i] = dfl[i] - (a*((angle[i]*math.pi)/180)+b)
  #print(dfl)
  return dfl

In [ ]:
#create a dataframe to store baseline corrected data
databaseline = pd.DataFrame(index=range(len(data_act)),columns=range(72))

In [ ]:
#perform baseline correction and add it to the dataframe
for i in range(data_act.shape[0]):
  l = []
  for j in data_act.iloc[i,:72] :
    l.append(j)
  databaseline.iloc[i,:] = linear(l)

In [ ]:
#transfer additional columns to the baseline corrected dataframe and perform validation (following cells)
databaseline['labels'] = data_act['labels']
databaseline['file'] = data_act['file']

In [ ]:
databaseline.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,labels,file
0,0.0,-0.003948,-0.005745,-0.013974,-0.017291,-0.019094,-0.010199,-0.001974,0.001256,-0.00266,...,-0.015105,-0.00508,0.00092,0.01481,-0.015582,0.012836,-0.002374,0.0,L_Proline,feb7
1,0.0,-0.023946,0.005688,-0.004492,0.002285,0.021508,0.025094,0.003451,0.021539,0.031765,...,0.016556,0.024326,0.048935,0.037194,0.044587,0.015952,0.017479,0.0,L_Proline,feb7
2,0.0,0.003637,0.01606,0.011741,0.012065,0.013699,0.017344,0.011104,0.013883,0.027348,...,-0.007926,-0.001473,-0.007281,0.013418,0.002928,0.023828,0.010745,0.0,L_Proline,feb7
3,0.0,-0.009657,0.024342,0.012211,-0.007716,-0.013797,0.00194,0.031969,0.009083,0.041681,...,-0.013641,-0.016722,-0.020625,-0.015813,-0.012739,-0.01328,-0.002411,0.0,L_Proline,feb7
4,0.0,-0.014923,0.002318,-0.020632,-0.020439,-0.043308,-0.01339,-0.028657,-0.002323,0.002113,...,0.013979,0.005283,0.010295,0.017501,0.042545,0.03977,0.015187,0.0,L_Proline,feb7


In [ ]:
databaseline.tail()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,labels,file
674,0.0,-0.015577,-0.013812,-0.01031,0.026628,-0.027949,-0.016891,0.01655,0.012233,0.008116,...,0.002194,0.019316,0.029078,0.016589,0.035936,0.020759,0.018963,0.0,L_Arginine,may3
675,0.0,-0.000738,0.02555,0.02378,0.002872,0.002078,0.002196,0.009164,0.024224,0.023542,...,0.013821,0.024054,0.02422,0.026396,0.035845,0.039941,0.008902,0.0,L_Arginine,may3
676,0.0,-0.009058,0.00494,0.000916,-0.023581,-0.007685,-0.00185,-0.011124,0.011294,0.020914,...,0.004843,-0.000101,0.006374,0.010601,0.008249,0.008851,0.005474,0.0,L_Arginine,may3
677,0.0,-0.005092,-0.019067,-0.011256,-0.01128,-0.01297,-0.006377,0.004099,0.000505,0.008587,...,0.000159,0.005761,0.004375,0.011824,0.009799,0.005837,0.005684,0.0,L_Arginine,may3
678,0.0,0.002551,-0.005461,-0.004301,-0.017531,0.003981,-0.002006,0.014886,0.015527,0.02188,...,-0.027837,-0.032569,-0.037836,-0.022718,-0.002758,0.012469,0.015063,0.0,L_Arginine,may3


In [ ]:
databaseline['file'].unique()

array(['feb7', 'Feb21', 'old', 'March3', 'March13', 'March10', 'March20',
       'mar24', 'mar29', 'april17', 'may3'], dtype=object)

In [ ]:
databaseline['labels'].unique()

array(['L_Proline', 'L_Lysine', 'L_Glutamine', 'L_Arginine', 'L_Serine',
       'L_Histidine', 'L_Threonine', 'L_Asparagine', 'L_Phenylalanine',
       'L_Alanine', 'L_Glutamate', 'L_Methionine', 'L_Cystine',
       'L_Valine', 'L_Isoleucine', 'L_Tryptophan', 'L_Leucine'],
      dtype=object)

In [ ]:
databaseline.shape

(679, 74)

In [ ]:
#save the data
databaseline.to_csv('/content/drive/My Drive/ED_AA_AI/spring23_dataprocessed/23_may3/may3fullbaselinelinearnotnormalized.csv', index = False)